In [4]:
# initial settting
from __future__ import print_function
import os
os.chdir("/usr/local/spark-2.3.0/")

## Dataset vs Dataframe
* Dataset: a distributed collection of data
    * only in scala and java 
    * map, flatMap, filter ....
* DataFrame: a _Dataset_ organized into named columns

## Getting Started
### SparkSession

In [3]:
%cat ./examples/src/main/python/sql/basic.py

#
# Licensed to the Apache Software Foundation (ASF) under one or more
# contributor license agreements.  See the NOTICE file distributed with
# this work for additional information regarding copyright ownership.
# The ASF licenses this file to You under the Apache License, Version 2.0
# (the "License"); you may not use this file except in compliance with
# the License.  You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#

"""
A simple example demonstrating basic Spark SQL features.
Run with:
  ./bin/spark-submit examples/src/main/python/sql/basic.py
"""
from __future__ import print_function

# $example on:init_session$
from pyspark.s

In [5]:
# launching spark session
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("hello-world") \
        .getOrCreate()
       #.config("key","value")

### Creating DataFrame

In [6]:
df = spark.read.json("./examples/src/main/resources/people.json")
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



### Untyped Dataset Operation

In [7]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [8]:
df.select('name').show()

+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
+-------+



In [9]:
df.select(df['name'],df['age']+1).show()

+-------+---------+
|   name|(age + 1)|
+-------+---------+
|Michael|     null|
|   Andy|       31|
| Justin|       20|
+-------+---------+



In [10]:
df.filter(df['age']>21).show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



In [11]:
df.groupBy("age").count().show()

+----+-----+
| age|count|
+----+-----+
|  19|    1|
|null|    1|
|  30|    1|
+----+-----+



### Running SQL Queris Programmatically

In [16]:
df.createOrReplaceTempView('people')
sqlDF = spark.sql("SELECT * FROM people")
sqlDF.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



### Global Temporary View

In [17]:
df.createOrReplaceGlobalTempView('people')
spark.sql("SELECT * FROM global_temp.people").show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [19]:
# global temporary view can be used in another session 
# which are different from the session who make the global temporary view
spark.newSession().sql("SELECT * FROM global_temp.people").show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



### InterOperating with RDDs
RDD의 스키마 추론하는 방법
* reflection: schema 오브젝트를 만들지 않고 넣기
* schema 오브젝트를 만들어서 넣기
#### reflection 이용, 스키마 추론하기

In [20]:
from pyspark.sql import Row
sc = spark.sparkContext # for RDD operations

In [22]:
lines = sc.textFile("examples/src/main/resources/people.txt")
parts = lines.map(lambda l: l.split(",")) # 한 줄을 ,로 나누기
people = parts.map(lambda p: Row(name=p[0], age=int(p[1]))) # 첫 컬럼은 이름, 두 번쨰 컬럼은 나이

In [24]:
# people은 RDD 형태이므로 이것을 DataFrame으로 변환
schemaPeople = spark.createDataFrame(people)
schemaPeople.createOrReplaceTempView("people")

In [27]:
# Using DataFrame API
teenagers = spark.sql("SELECT name FROM people WHERE age >= 13 AND age <= 19")
teenagers.show()

+------+
|  name|
+------+
|Justin|
+------+



In [28]:
# Using RDD API
teenagers.rdd.map(lambda p: "Name: "+p.name).collect() # p는 dict 형태로 들어오는 것 같음

[u'Name: Justin']

#### Programmatically하게 스키마 명시하기(직접 찾기)

In [29]:
# type import
from pyspark.sql.types import *

In [30]:
# using the same lines, parts, and people
# making schema
schemaString = "name age"

# list of StructField
fields = [StructField(field_name, StringType(), True) for field_name in schemaString.split()]
# create StructType object using the list of StructField created above
schema = StructType(fields)

schemaPeople = spark.createDataFrame(people, schema) # schema 직접 명시

# Creates a temporary view using the DataFrame
schemaPeople.createOrReplaceTempView("people")

# SQL can be run over DataFrames that have been registered as a table.
results = spark.sql("SELECT name FROM people")
results.show()

+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
+-------+



## Data Load/ Save
[문서](https://spark.apache.org/docs/latest/sql-programming-guide.html#data-sources) 참고

## PySpark Usage Guide for Pandas with Apache Arrow
### PyArrow install
```bash
# conda
conda install -c conda-forge pyarrow

#pip
pip install pyarrow
```
### Enabling for vonersion to/from Pandas
Spark DataFrame --> Pandas DataFrame by calling `toPandas()`

Spark configuration `spark.sql.execution.arrow.enabled` must be `true`

In [40]:
import numpy as np
import pandas as pd

# Enable Arrow-based columnar data transfers
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

# Generate a Pandas DataFrame
pdf = pd.DataFrame(np.random.rand(100, 3))

# Create a Spark DataFrame from a Pandas DataFrame using Arrow
df = spark.createDataFrame(pdf) # if arrow is not enabled, it raise error.

# Convert the Spark DataFrame back to a Pandas DataFrame using Arrow
result_pdf = df.select("*").toPandas()

In [41]:
result_pdf

,0,1,2
0,0.324552,0.804630,0.990966
1,0.937478,0.256441,0.642096
2,0.088825,0.364415,0.950739
3,0.953102,0.468424,0.437852
4,0.645455,0.636838,0.429910
5,0.142951,0.922369,0.761218
6,0.831337,0.980546,0.465130
7,0.985369,0.404239,0.458919
8,0.192842,0.372941,0.190815
9,0.626144,0.584046,0.879541


### Pandas UDF( aka Vectorized UDFs)

Pandas UDF는 Spark가 Arrow를 이용해서 데이터를 전송하여 Pandas가 데이터를 처리할 수 있도록 하는 user defined 함수이다. `pandas_udf`라는 데코레이터를 붙이거나 함수를 wrap함으로써 정의할 수 있다.

현재는 2 가지의 Pandas UDF가 있다.
* Scalar
* Grouped Map

#### Scalar
벡터화된 스칼라 연산에 사용된다. `select`나 `withColumn` 함수와 같이 사용될 수 있다. 내부적으로 Spark는 컬럼들을 배치로 나누고 각 배치마다 Pandas UDF를 호출한 후 다시 결합하여 Pandas UDF를 처리한다.

In [43]:
import pandas as pd

from pyspark.sql.functions import col, pandas_udf
from pyspark.sql.types import LongType

In [44]:
# Declare the function and create the UDF
def multiply_func(a, b):
    return a * b
multiply = pandas_udf(multiply_func, returnType=LongType())

In [45]:
# pandas에서 계산
x = pd.Series([1, 2, 3])
print(multiply_func(x, x))

0    1
1    4
2    9
dtype: int64


In [46]:
# Spark DataFrame에서 pandas udf를 호출하여 계산
# Create a Spark DataFrame, 'spark' is an existing SparkSession
df = spark.createDataFrame(pd.DataFrame(x, columns=["x"]))
# Execute function as a Spark vectorized UDF
df.select(multiply(col("x"), col("x"))).show()

+-------------------+
|multiply_func(x, x)|
+-------------------+
|                  1|
|                  4|
|                  9|
+-------------------+



#### Grouped Map
`groupBy().apply()`와 같이 사용된다.

* `DataFrame.groupBy`를 통해 데이터를 그룹을 나눈다.
* 각 그룹마다 함수를 적용한다. 입력값과 출력값 모두 `pandas.DataFrame`이다.
* 결과들을 합쳐 새로운 DataFrame을 만든다.

`groupBy().apply()`를 사용하기 위해 다음 2개를 정의해야 한다.
* 각 그룹별로 계산에 사용될 파이썬 함수
* 출력할 `DataFrame`의 스키마가 정의된 `StructType` 객체나 스트링

출력 스키마는 리턴하는 pandas.DataFrame의 컬럼의 이름이 아닌 위치에 따라 적용되므로 컬럼들이 반드시 인덱싱이 되어 있어야 한다.

그룹 사이즈가 skewed되어 있으면 어떤 노드에서는 메모리가 부족할 수 있다. `maxRecordsPerBatch`는 그룹에는 적용되지 않으므로 유저가 알아서 그룹을 잘 분배해야 한다.

In [49]:
from pyspark.sql.functions import pandas_udf, PandasUDFType
df = spark.createDataFrame(
    [(1, 1.0), (1, 2.0), (2, 3.0), (2, 5.0), (2, 10.0)],
    ("id", "v"))

In [50]:
@pandas_udf("id long, v double", PandasUDFType.GROUPED_MAP)
def substract_mean(pdf):
    # pdf is a pandas.DataFrame
    v = pdf.v
    return pdf.assign(v=v - v.mean())

In [51]:
df.groupby("id").apply(substract_mean).show()

+---+----+
| id|   v|
+---+----+
|  1|-0.5|
|  1| 0.5|
|  2|-3.0|
|  2|-1.0|
|  2| 4.0|
+---+----+

